## Assign path

In [1]:
# root의 경우 ''안을 파일들이 들어있는 폴더의 경로로 변경합니다.
# 만약 root 폴더가 D:\chinese\data/train3\data인 경우, 해당 폴더 안은 아래에 첨부한 사진과 같은 형식으로 존재하게 해주시면 됩니다.
# save_path의 경우 ''안에 전처리된 데이터를 저장할 파일의 이름을 정하여 작성해주시면 됩니다.
# 경로를 지정해주시고, Preprocess data, Check preprocessed file 아래의 내용까지 돌려주시면 됩니다. (돌리고 싶은 칸을 누르고 shift+enter)

root = 'D:\chinese\data/train3\data'
save_path = 'our_train3.txt' 

<img src="folder.png" width="800" height="400">

## Preprocess data
This is code for preprocessing data.
You can preprocess data by excuting the code followed.


In [2]:
import os 
from docx import Document
import re
import numpy as np
from difflib import get_close_matches
# Make the file list
cleans = []
puncs = []
for i in os.listdir(root):
    cleans = cleans + [os.path.join(root,i,'clean',j) for j in os.listdir(os.path.join(root, i, 'clean'))]
    puncs = puncs + [os.path.join(root,i,'punc',j) for j in os.listdir(os.path.join(root, i, 'punc'))]
assert len(cleans)==len(puncs)

In [3]:
def load_text(f):
    # load file
    text = Document(f)
    data = ''
    # remove title
    i=0
    while len(text.paragraphs[i].text) <=1:
        i+=1
    for j in text.paragraphs[i+1:]:
        if len(j.text)>2:
            data = data + j.text

    # replace undefined punctuation to defined punctuation
    data = data.replace("〈", "《").replace("〉", "》").replace('（', '(').replace('）',')').replace('+','')
    data = data.replace('【','(').replace('】', ')').replace('[', '(').replace(']', ')').replace('〔', '(').replace('〕', ')')
    data = data.replace('《 文摘报 》', '').replace('文摘报', '').replace('■', '').replace('/','').replace('~','').replace('●','')
    data = data.replace(' ', '').replace('　', '').replace('\u3000', '').replace('\xa0', '').replace('\n','')
    data = data.replace('‘','“').replace('’','”').replace('？', '?').replace('！', '!').replace("，", ",").replace("：", ":").replace("；", ";")

    return data

def change_format(t1, t2):
    # compare clean file and punctuation file
    # make the input and label
    num = 0
    while t1[0] != t2[num]:
        num += 1
    data = ''
    for i in range(len(t1)):
        data+=t1[i]+'\t'
        num+=1
        if i == len(t1)-1:
            data+=t2[num:]+'\n'
        elif t1[i+1] == t2[num]:
            data+='O\n'
        else:
            while t1[i+1] != t2[num]:
                data+=t2[num]
                num+=1
            data+='\n'

    # remove () and inside
    data = re.sub(pattern=r'O\n\([^)]*\)\t', repl='', string=data)
    data = re.sub(pattern=r'\n\([^)]*\)\t', repl='', string=data)
    data = re.sub(pattern=r'\t\([^)]*\)\n', repl='\tO\n', string=data)
    data = re.sub(pattern=r'\t\([^)]*\)', repl='\t', string=data)
    data = re.sub(pattern=r'\([^)]*\)\n', repl='\n', string=data)
    data = re.sub(pattern=r'\([^)]*\)', repl='', string=data)
    if data.find(')')>0:
        data = data[data.find(')')+1:]
        idx = data.index('\n')
        data = data[idx+1:]
        
    return data

In [4]:
f = open(save_path, 'w', encoding='utf-8')
for i in puncs:
    try:
        clean = get_close_matches(i, cleans, 1, 0.9)
        text1 = load_text(os.path.join(root, 'clean', clean[0]))
        text2 = load_text(i)
        f.write(change_format(text1, text2))
    except Exception as e:        
        print(i)
        # 문장부호가 있는 파일과 없는 파일에서 형식이나 기호가 서로 다른 내용이 존재해 에러가 나오는 파일의 경우 제외하였습니다.
        # 아래 나오는 파일명은 제외된 파일입니다.
f.close()

D:\chinese\data/train3\data\200\punc\158.괏앎撚괏藤낀狼吝路懇끝방쪼.docx
D:\chinese\data/train3\data\200\punc\200.櫓빽匡츠돨愷몸景醴.docx
D:\chinese\data/train3\data\200\punc\36.“무列竟”呵혼懇끝 숨聯벌췽懃檄裂긴.docx
D:\chinese\data/train3\data\200\punc\80.“5怯출옘쏵굇댕”，싱竿랙喇뎠쏴.docx
D:\chinese\data/train3\data\500（1）\punc\134.벌소寮柩構쐤틱랙깊랗찖랗寧쾨劤쾨붕늦.docx
D:\chinese\data/train3\data\500（1）\punc\165.構쐤틱瞳젬북벌膠뜩湳昑룟삔돨쉿뺐.docx
D:\chinese\data/train3\data\500（1）\punc\217.벌소寮柩構쐤틱랙깊랗찖랗찖쾨劤쾨붕늦.docx
D:\chinese\data/train3\data\500（1）\punc\221.構쐤틱밑黨《櫓묾櫓卵밑黨솽넣뵨供櫓벌景삔寮屢齡똑 股쏵벌소撈잿竟溝뵨撈잿콘제君덜뺏흼맴路댕狂痙돨엄땍》돨綱츠.docx
D:\chinese\data/train3\data\500（1）\punc\245.構쐤틱瞳랗찖寧씽쾨뉴쌘考격삔돨쉿뺐.docx
D:\chinese\data/train3\data\500（1）\punc\323.벌소寮柩構쐤틱랙깊랗찖寧펌쾨劤쾨붕늦.docx
D:\chinese\data/train3\data\500（1）\punc\340.構쐤틱瞳欺構《빚쏩勍匡朞》괩멩삔돨쉿뺐（1）.docx
D:\chinese\data/train3\data\500（1）\punc\341.構쐤틱瞳欺構《빚쏩勍匡朞》괩멩삔돨쉿뺐（2）.docx
D:\chinese\data/train3\data\500（1）\punc\383.構쐤틱：瞳삔숨홍벌膽汲群官蝎션珂돨쉿뺐.docx
D:\chinese\data/train3\data\500（1）\punc\394.벌소寮柩構쐤틱랙깊랗찖寧巧쾨劤쾨붕늦.docx
D:\chinese\data/train3\dat

## Check preprocessed file
Following code check the preprocessed file and fix the error.   
If something that is not a punctuation mark appears, check the preprocessed file and correct it.

In [5]:
f = open(save_path, 'r', encoding='utf-8') # preprocessed file path
data = f.readlines()
f.close()
data2 = []
for i in range(len(data)):
    data[i] = data[i].replace('\u3000', '').replace(' ', '').replace('  ', '').replace('\xa0','').replace("\u2003", '')
    data[i] = data[i].replace(',', '，').replace('......', '……').replace('.','。').replace('-', '—').replace("─", '—').replace("一", '—').replace('"', 'O')
    try:
        if data[i].split('\t')[1] == '\n':
            data[i] = data[i][:-1]+'O\n'
        data2.append(data[i])
    except:
        continue
    
punc = [i.split('\t')[1][0] for i in data2]
print(set(punc))

f = open(save_path, "w", encoding='utf-8')
for i in data2:
    f.write(i.split('\t')[0]+'\t'+i.split('\t')[1][0]+'\n')
f.close()

{'《', '交', '·', '国', '方', '、', '电', '》', ')', '“', '美', '3', '1', '…', '?', 'O', ':', '陈', '创', '等', ';', '。', '，', '”', '研', '月', '!', '2', '—'}


In [6]:
# 여기서 결과로 출력되는 것 중 문장 부호가 아닌 것은 직접 전처리 결과로 나온 파일에 들어가 수정해줍니다.
# 전처리 파일은 현재 파일이 있는 폴더에 가장 상단에서 save_path에 작성한 이름으로 생성되었을 것입니다.
# 전처리 파일을 보면, 왼쪽에 한자, 오른쪽에 문장부호 혹은 O가 존재하는데, 여기서 나온 한자나 사용하지 않는 문장부호를 O로 수정하였습니다.
# 수정 후, 아래의 Count punc를 실행하시면 각 문장부호가 데이터에서 몇번 등장하는지 확인할 수 있습니다.

### Count punc
Count the number of punctuation in each dataset.

In [7]:
import pandas as pd
import numpy as np

def count_punctuation(path):
    f = open(path, 'r', encoding='utf-8')
    data = f.readlines()
    data = [i.split('\t')[1][0] for i in data]
    count = [data.count(','), data.count('。'), data.count('!'), data.count('?'), data.count(';'), data.count(':'), data.count('“'), data.count('”'),
            data.count('…'), data.count('—'), data.count('、'), data.count('·'), data.count('《'), data.count('》')]
    result = pd.DataFrame(
        np.array([count]), 
        columns=list([',', '。', '!', '?', ';', ':', '“', '”', '…', '—', '、', '·', '《', '》']),
        index = ['Count']
    )

    return result

In [8]:
path = 'D:\chinese_punctuation\data\our_test.txt'
count_punctuation(path) # path to dataset what you want to count

,",",。,!,?,;,:,“,”,…,—,、,·,《,》
Count,9040,3702,28,171,155,227,1041,1330,47,74,2246,41,304,457
